In [13]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

## Import migration background per neigbourhood in Amsterdam (raw data)

In [15]:
url = "https://api.data.amsterdam.nl/dcatd/datasets/JAVeX44oiXm_5Q/purls/5"
dfcord = pd.read_excel(url)
dfcord[1:10]

,"1.5a Bevolking buurten naar migratieachtergrond, 1 januari 2019",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
1,buurt,Surinaams,Antilliaans,Turks,Marokkaans,overig niet-westers,totaal niet-westers,westers,Nederlands,totaal
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00a Kop Zeedijk,25,8,11,3,132,179,334,522,1035
4,A00b Oude Kerk e.o.,14,7,7,4,82,114,247,343,704
5,A00c Burgwallen Oost,32,18,14,17,213,294,513,766,1573
6,A00d Nes e.o.,2,2,3,1,28,36,122,194,352
7,A00e BG-terrein e.o.,12,5,5,10,65,97,173,359,629
8,A01a Stationsplein e.o.,-,-,-,-,-,-,2,-,2
9,A01b Hemelrijk,15,7,4,4,74,104,229,270,603


In [16]:
dfcord.shape

(487, 10)

## import employment per sector per neighbourhood in Amsterdam

In [17]:
url2 = "https://api.data.amsterdam.nl/dcatd/datasets/-fotdSpwjmSK9Q/purls/3" 
dfsectors = pd.read_excel(url2)
dfsectors[1:10]

,"2.3 Werkzame personen buurten naar hoofdfunctie, 1 januari 2018 1)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1,buurt,kantoren,winkels,voorzieningen,horeca,bedrijven,overig,totaal
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00a Kop Zeedijk,187,135,91,912,98,35,1458
4,A00b Oude Kerk e.o.,742,866,80,817,44,83,2632
5,A00c Burgwallen Oost,392,213,240,603,318,115,1881
6,A00d Nes e.o.,1058,391,100,536,126,*,2215
7,A00e BG-terrein e.o.,298,75,1341,636,75,8,2433
8,A01a Stationsplein e.o.,61,190,*,317,1703,*,2348
9,A01b Hemelrijk,331,106,158,364,159,11,1129


In [18]:
dfsectors.shape

(487, 8)

## Show Amsterdam

In [23]:
# The code was removed by Watson Studio for sharing.

Your credentails:


In [24]:
latitude = 52.379189
longitude =  4.899431
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

In [25]:
venues_map